In [1]:
# 2. Implement PPO

In [56]:
# !pip install gymnasium

In [57]:
# !pip install swig "gymnasium[box2d]"

In [4]:
import gymnasium as gym

In [5]:
env = gym.make('Pendulum-v1', render_mode='human')

In [6]:
env.action_space

Box(-2.0, 2.0, (1,), float32)

In [7]:
env.action_space.shape

(1,)

In [8]:
env.observation_space.shape

(3,)

In [12]:
env.observation_space

Box([-1. -1. -8.], [1. 1. 8.], (3,), float32)

In [17]:
env.action_space.sample()

array([1.2128432], dtype=float32)

In [55]:
# for i in range(1000):
#     print(env.step([0]))

In [42]:
# # Reset the environment to generate the first observation
# observation, info = env.reset(seed=42)
# for _ in range(1000):
#     # this is where you would insert your policy
#     action = env.action_space.sample()

#     # step (transition) through the environment with the action
#     # receiving the next observation, reward and if the episode has terminateda or truncated
#     observation, reward, terminated, truncated, info = env.step(action)

#     # If the episode has ended then we can reset to start a new episode
#     if terminated or truncated:
#         observation, info = env.reset()

# env.close()

In [76]:
env.reset()

(array([-0.00236187,  1.413665  , -0.23925176,  0.12199411,  0.00274366,
         0.05419413,  0.        ,  0.        ], dtype=float32),
 {})

In [77]:
env.render()

In [81]:
env.action_space

Box(-2.0, 2.0, (1,), float32)

In [58]:
env.observation_space

Box([-1. -1. -8.], [1. 1. 8.], (3,), float32)

In [89]:
timesteps_per_batch = 1000
max_ep_len = 1000

In [90]:
# simulate one episode up to max length

In [91]:
observation, info = env.reset(seed=42)

In [92]:
observation

array([-0.14995256,  0.9886932 , -0.12224312], dtype=float32)

In [94]:
info

{}

In [95]:
action = env.action_space.sample()

In [97]:
action

array([-0.9419495], dtype=float32)

In [96]:
observation, reward, terminated, truncated, info = env.step(action)

In [100]:
terminated

False

In [101]:
truncated

False

In [103]:
info

{}

In [ ]:
# observation, info = env.reset(seed=42)
# for _ in range(1000):
#     # this is where you would insert your policy
#     action = env.action_space.sample()

#     # step (transition) through the environment with the action
#     # receiving the next observation, reward and if the episode has terminateda or truncated
#     observation, reward, terminated, truncated, info = env.step(action)

#     # If the episode has ended then we can reset to start a new episode
#     if terminated or truncated:
#         observation, info = env.reset()

# env.close()

In [108]:
env.reset()[0].shape

(3,)

In [143]:
import datetime

In [151]:
class PPO: # asked ChatGPT to give me the class skeleton
    def __init__(self, policy_cls, env):
        self.obs_dim = env.observation_space.shape[0]
        self.act_dim = env.action_space.shape[0]
        self.actor = policy_cls(self.obs_dim, self.act_dim)
        self.critic = policy_cls(self.obs_dim, 1) # for the reward
        self.cov_mat = torch.eye(env.action_space.shape[0])
        self.max_timesteps = 200
        
    def get_action(self, obs):
        mean = self.actor.forward(torch.tensor(obs))
        dist = MultivariateNormal(mean, self.cov_mat)
        action = dist.sample()
        return dist.sample(), dist.log_prob(action)
    
    def rollout(self, timesteps_per_batch):
        batch_obs = []
        batch_acts = []
        batch_log_probs = []
        
        batch_lens = []
        batch_rews = []
        batch_rtgs = []
        collected = 0
        
        while collected < timesteps_per_batch:
            print(datetime.datetime.now())
            print(collected)
            ep_t = 0 
            ep_rews = []
            obs, _ = env.reset()
            terminated, truncated = False, False
            while not terminated and not truncated and ep_t < self.max_timesteps:
                act, log_prob = self.get_action(obs)
                obs, rew, terminated, truncated, _ = env.step(act)
                batch_obs.append(obs)
                batch_acts.append(act)
                batch_log_probs.append(log_prob)
#                 rtgs = # consulted 
#                 batch_lens = 
                ep_t += 1
                ep_rews.append(rew)
                collected += 1
            batch_lens.append(ep_t + 1)
            batch_rews.append(ep_rews)
            
        batch_obs = torch.tensor(batch_obs, dtype=torch.float)
        batch_acts = torch.tensor(batch_acts, dtype=torch.float)
        batch_log_probs = torch.tensor(batch_log_probs, dtype=torch.float)
        batch_rtgs = self.compute_rtgs(batch_rews)
        
        return batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens
            
    @staticmethod # learned something new here
    def compute_rtgs(batch_rews, gamma): # consulted ChatGPT
        
        
        
    
    
#     def learn(self, env, n_timesteps = 1000):
#         for t in range(n_timesteps):
            
            

In [152]:
import torch
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions.normal import Normal

In [153]:
model = PPO(FeedForwardNN, env)

In [154]:
obs, _ = env.reset()

In [155]:
model.get_action(obs)

(tensor([2.3715]), tensor(-1.0040, grad_fn=<SubBackward0>))

In [156]:
model.rollout(1000)

2025-06-03 22:01:09.888966
0
2025-06-03 22:01:16.708343
200
2025-06-03 22:01:23.516078
400
2025-06-03 22:01:30.336312
600
2025-06-03 22:01:37.160041
800


(tensor([[-0.1647, -0.9863, -0.4069],
         [-0.2210, -0.9753, -1.1479],
         [-0.3102, -0.9507, -1.8509],
         ...,
         [-0.1353, -0.9908, -1.6518],
         [-0.2495, -0.9684, -2.3291],
         [-0.3943, -0.9190, -3.0615]]),
 tensor([-2.9076e-02, -8.3482e-03,  1.8953e-01, -1.3003e+00, -5.9516e-01,
         -4.5322e-02,  2.2490e+00, -9.2591e-01, -1.2165e+00,  8.3836e-01,
          4.3991e-02,  2.8383e-01,  8.5482e-01,  5.2608e-01,  5.6938e-01,
          7.4281e-01,  2.1864e-01,  8.6609e-01, -5.5831e-01, -1.7975e+00,
          3.7256e-01, -1.8093e+00,  1.8011e+00,  4.2690e-01,  2.0764e-01,
          3.0229e-01, -3.9420e-01,  9.4598e-02,  6.5420e-01, -7.0653e-01,
         -3.1216e-01,  1.5456e+00, -4.0357e-01,  9.7965e-01, -7.7600e-01,
         -1.7789e+00, -1.8941e+00,  2.9155e-01, -2.4498e-03,  7.2697e-01,
         -2.4961e-01,  1.9850e+00, -1.5172e-01, -9.7602e-01, -1.0015e+00,
         -2.7496e-01,  1.1058e+00, -5.6173e-01, -1.6438e+00, -3.0378e-01,
         -5.8729

In [116]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

In [117]:
class FeedForwardNN(nn.Module):
    def __init__(self, in_dim, out_dim): # got in_dim hint
        super(FeedForwardNN, self).__init__()
        self.layer1 = nn.Linear(in_dim, 64) # got nn.Linear hint
        self.layer2 = nn.Linear(64, 64)
        self.layer3 = nn.Linear(64, out_dim)
    def forward(self, x):
        x = F.relu(self.layer1(x)) # NOT nn.ReLU - which is the actual layer
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [118]:
# testNN = FeedForwardNN(5,6)

In [119]:
# testNN(torch.tensor([1,2,3,4,5]).to(torch.float32))